# 3D Phase Space Manifold Visualization

## Detailed Analysis

This script is used to visualize the topological separation between Hadronic and Quark stars in the full three-dimensional space of observables: Mass ($M$), Radius ($R$), and Tidal Deformability ($\Lambda$).

The visualization produces two outputs: a static PDF for the thesis document and an interactive HTML file (using Plotly) for exploration. A key feature of these plots is the inclusion of "shadow" projections on the axis walls. These projections show the density of the populations in 2D subspaces (e.g., the standard Mass-Radius plane), linking the 3D manifold structure to familiar 2D physical relations.

## Physics and Math

The classification of neutron stars relies on the separation of their Equations of State in the observable parameter space $\mathcal{S}$:

$$
\mathcal{S} = \{ R, \, M, \, \log_{10}(\Lambda) \}
$$

### Dimensionality Reduction (Projections)
While the full dataset exists in 3D, physical constraints are often studied in 2D marginal distributions. The script computes these projections using Kernel Density Estimation (KDE). For a set of points $(x_i, y_i)$, the probability density function $P(x,y)$ is estimated and projected onto a plane:

$$
P_{proj}(x, y) = \int P(x, y, z) \, dz
$$

This allows the viewer to see the "shadow" of the 3D data, effectively reproducing the Mass-Radius relation, the Mass-Tidal relation, and the Radius-Tidal relation on the walls of the 3D bounding box.

## Code Walkthrough

### 1. Data Preparation
The script first checks for the existence of the logarithmic tidal deformability feature. If it is missing, it is calculated from the raw $\Lambda$:

```python
if 'LogLambda' not in df.columns:
    df['LogLambda'] = np.log10(df['Lambda'])
```
The data is then split into Hadronic (Label 0) and Quark (Label 1) subsets. Limits for the axes are loaded from `src.const.CONSTANTS` to ensure consistency with other figures.

### 2. Density Grid Calculation
A helper function, `get_density_grid`, is defined to compute the KDE for the wall projections.
*   It accepts x and y data arrays.
*   It creates a meshgrid based on the plot limits.
*   It computes the Gaussian KDE and normalizes the result for plotting as contours.

### 3. Static 3D Plotting (Matplotlib)
The script uses `mpl_toolkits.mplot3d` to generate the static figure.

*   **Scatter Plot:** A subset of points (e.g., 3000) is sampled and plotted as semi-transparent markers to visualize the core "cloud" of the data.
*   **Wall Projections:** The `plot_mpl_wall` function is used to draw the calculated KDE contours on the $xy$, $xz$, and $yz$ planes (offset to the axis limits).

```python
# Floor (z=0): Radius vs Mass
plot_mpl_wall(ax, h_data['Radius'], h_data['Mass'], ..., 'z', l_lim[0], ...)
```
This renders the familiar Mass-Radius relation on the "floor" of the plot.

### 4. Interactive 3D Plotting (Plotly)
If the `plotly` library is available, an interactive HTML file is generated.
*   **Scatter3d:** The main data points are added as interactive markers.
*   **Surface Traces:** The KDE grids are converted into 3D surfaces and placed on the bounding box walls.
*   **Z-Fighting Fix:** Slight offsets are applied to the wall surfaces to ensure that the Hadronic and Quark layers do not flicker when rendered on top of each other.

```python
if wall_type == 'floor':
    z_h = np.full_like(XX, l_lim[0])
    z_q = np.full_like(XX, l_lim[0] + 0.02) # Small offset
```

## Visualization Output

The script generates `plots/fig_12_3d_manifold.pdf` (and optionally an HTML file).

*   **X-Axis:** Radius $R$ [km].
*   **Y-Axis:** Mass $M$ [$M_{\odot}$].
*   **Z-Axis:** Log Tidal Deformability $\log_{10}\Lambda$.
*   **Visual Elements:**
    *   **Scatter Cloud:** Shows individual star configurations. Hadronic stars (Green) typically form a distinct branch from Quark stars (Magenta), especially at high masses or low radii.
    *   **Wall Shadows:**
        *   **Floor ($R-M$):** Shows the standard Mass-Radius relation.
        *   **Back Wall ($R-\Lambda$):** Shows the strong correlation between Radius and Tidal Deformability.
        *   **Side Wall ($M-\Lambda$):** Shows how deformability decreases rapidly as mass increases.
*   **Colors:** Hadronic stars are Green; Quark stars are Magenta.